In [21]:
!pip install rouge_score

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/133.7 kB ? eta -:--:--
   ------------ --------------------------- 41.0/133.7 kB 2.0 MB/s eta 0:00:01
   ------------------ --------------------- 61.4/133.7 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------  133.1/133.7 kB 1.1 MB/s eta 0:00:01
   -------------------------------------- 133.7/133.7 kB 987.7 kB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24970 sha256=78b89b06dab3859b261d883b6aabb57c011069b3ffe569598e125d9819292ad6
  Stored in directory: c:\users\joshl\appdata\local\pip\cache\wheels\1e\19\43\8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [38]:
context = """Instruction tuning large language models (LLMs) using machine-generated
instruction-following data has been shown to improve zero-shot capabilities on
new tasks, but the idea is less explored in the multimodal field. We present the
first attempt to use language-only GPT-4 to generate multimodal language-image
instruction-following data. By instruction tuning on such generated data, we introduce LLaVA: Large Language and Vision Assistant, an end-to-end trained
large multimodal model that connects a vision encoder and an LLM for generalpurpose visual and language understanding. To facilitate future research on visual
instruction following, we construct two evaluation benchmarks with diverse and
challenging application-oriented tasks. Our experiments show that LLaVA demonstrates impressive multimodal chat abilities, sometimes exhibiting the behaviors
of multimodal GPT-4 on unseen images/instructions, and yields a 85.1% relative score compared with GPT-4 on a synthetic multimodal instruction-following
dataset. When fine-tuned on Science QA, the synergy of LLaVA and GPT-4
achieves a new state-of-the-art accuracy of 92.53%. We make GPT-4 generated
visual instruction tuning data, our model, and code publicly available.
1 Introduction
Humans interact with the world through many channels such as vision and language, as each
individual channel has a unique advantage in representing and communicating certain concepts, and
thus facilitates a better understanding of the world. One of the core aspirations in artificial intelligence
is to develop a general-purpose assistant that can effectively follow multi-modal vision-and-language
instructions, aligned with human intent to complete various real-world tasks in the wild [4, 27, 26].
To this end, the community has witnessed an emergent interest in developing language-augmented
foundation vision models [27, 16], with strong capabilities in open-world visual understanding
such as classification [40, 21, 57, 54, 39], detection [29, 62, 33], segmentation [25, 63, 58] and
captioning [50, 28], as well as visual generation and editing [42, 43, 56, 15, 44, 30]. We refer readers
to the Computer Vision in the Wild reading list for a more up-to-date literature compilation [12]. In
this line of work, each task is solved independently by one single large vision model, with the task
instruction implicitly considered in the model design. Further, language is only utilized to describe
the image content. While this allows language to play an important role in mapping visual signals to
language semantics—a common channel for human communication, it leads to models that usually
have a fixed interface with limited interactivity and adaptability to the user’s instructions.
Large language models (LLM), on the other hand, have shown that language can play a wider
role: a universal interface for a general-purpose assistant, where various task instructions can be
explicitly represented in language and guide the end-to-end trained neural assistant to switch to the
task of interest to solve it. For example, the recent success of ChatGPT [35] and GPT-4 [36] have
demonstrated the power of aligned LLMs in following human instructions, and have stimulated
tremendous interest in developing open-source LLMs. Among them, LLaMA [49] is an opensource LLM that matches the performance of GPT-3. Alpaca [48], Vicuna [9], GPT-4-LLM [38]
37th Conference on Neural Information Processing Systems (NeurIPS 2023).
arXiv:2304.08485v2 [cs.CV] 11 Dec 2023
utilize various machine-generated high-quality instruction-following samples to improve the LLM’s
alignment ability, reporting impressive performance compared with proprietary LLMs. Importantly,
this line of work is text-only.
In this paper, we present visual instruction-tuning, the first attempt to extend instruction-tuning to
the language-image multimodal space, to pave the way towards building a general-purpose visual
assistant. In particular, our paper makes the following contributions:
• Multimodal instruction-following data. One key challenge is the lack of vision-language
instruction-following data. We present a data reformation perspective and pipeline to convert
image-text pairs into an appropriate instruction-following format, using ChatGPT/GPT-4.
• Large multimodal models. We develop a large multimodal model (LMM), by connecting the
open-set visual encoder of CLIP [40] with the language decoder Vicuna [9], and fine-tuning
end-to-end on our generated instructional vision-language data. Our empirical study validates
the effectiveness of using generated data for LMM instruction-tuning, and suggests practical
tips for building a general-purpose instruction-following visual agent. When ensembled with
GPT-4, our approach achieves SoTA on the Science QA [34] multimodal reasoning dataset.
• Multimodal instruction-following benchmark. We present LLaVA-Bench with two challenging
benchmarks, with a diverse selection of paired images, instructions and detailed annotations.
• Open-source. We release the following assets to the public: the generated multimodal instruction
data, the codebase, the model checkpoints, and a visual chat demo.
2 Related Work
Multimodal Instruction-following Agents. In computer vision, existing works that build instructionfollowing agents can be broadly categorized into two classes: (i) End-to-end trained models, which
are separately explored for each specific research topic. For example, the vision-language navigation
task [3, 19] and Habitat [47] require the embodied AI agent to follow natural language instructions
and take a sequence of actions to complete goals in visual environments. In the image editing domain,
given an input image and a written instruction that tells the agent what to do, InstructPix2Pix [6]
edits images by following the human instructions. (ii) A system that coordinates various models
via LangChain [1] / LLMs [35], such as Visual ChatGPT [53], X-GPT [63], MM-REACT [55],
VisProg [18], and ViperGPT [46]. While sharing the same goal in building instruction-following
agents, we focus on developing an end-to-end trained language-vision multimodal model for multiple
tasks.
Instruction Tuning. In the natural language processing (NLP) community, to enable LLMs such
as GPT-3 [7], T5 [41], PaLM [10], and OPT [60] to follow natural language instructions and
complete real-world tasks, researchers have explored methods for LLM instruction-tuning [37, 52, 51],
leading to instruction-tuned counterparts such as InstructGPT [37]/ChatGPT [35], FLAN-T5 [11],
FLAN-PaLM [11], and OPT-IML [22], respectively. It turns out that this simple approach can
effectively improve the zero- and few-shot generalization abilities of LLMs. It is thus natural
to borrow the idea from NLP to computer vision. More broadly, the teacher-student distillation
ideas with foundation models have been studied in other topics such as image classification [14].
Flamingo [2] can be viewed as the GPT-3 moment in the multimodal domain, due to its strong
performance on zero-shot task transfer and in-context-learning. Other LMMs trained on imagetext pairs include BLIP-2 [28], FROMAGe [24], and KOSMOS-1 [20]. PaLM-E [13] is an LMM
for embodied AI. Based on the recent “best” open-source LLM LLaMA, OpenFlamingo [5] and
LLaMA-Adapter [59] are open-source efforts that enable LLaMA to use image inputs, paving the
way to build open-source multimodal LLMs. While these models present promising task transfer
generalization performance, they are not explicitly tuned with vision-language instruction data, and
their performance in multimodal tasks usually falls short compared to language-only tasks. In this
paper, we aim to fill this gap and study its effectiveness. Finally, note that visual instruction tuning
is different from visual prompt tuning [23]: the former aims to improve the model’s instructionfollowing abilities, while the latter aims to improve the parameter-efficiency in model adaptation.
3 GPT-assisted Visual Instruction Data Generation
The community has witnessed a surge in the amount of public multimodal data such as image-text
pairs, ranging from CC [8] to LAION [45]. However, when it comes to multimodal instruction2
Context type 1: Captions
A group of people standing outside of a black vehicle with various luggage.
Luggage surrounds a vehicle in an underground parking area
People try to fit all of their luggage in an SUV.
The sport utility vehicle is parked in the public garage, being packed for a trip
Some people with luggage near a van that is transporting it.
Context type 2: Boxes
person: [0.681, 0.242, 0.774, 0.694], backpack: [0.384, 0.696, 0.485, 0.914], suitcase: ...<omitted>
Response type 1: conversation
Question: What type of vehicle is featured in the image?
Answer: The image features a black sport utility vehicle (SUV) ...<omitted>
Response type 2: detailed description
The image is an underground parking area with a black sport utility vehicle (SUV) parked. There
are three people in the scene, with one person standing closer to the left side of the vehicle, another
person in the middle, and the third person on the right side. They are all working together to pack
their luggage into the SUV for a trip. ...<omitted>
Response type 3: complex reasoning
Question: What challenges do these people face?
Answer: In the image, a group of people is standing outside a black SUV in a parking area,
surrounded by various pieces of luggage, including suitcases and backpacks. They are facing the
challenge of fitting all their luggage into the black SUV. There are multiple suitcases and backpacks
to be packed, which suggests that the group has a significant amount of belongings ...<omitted>
Table 1: One example to illustrate the instruction-following data. The top block shows the contexts
such as captions and boxes used to prompt GPT, and the bottom block shows the three types of
responses. Note that the visual image is not used to prompt GPT, we only show it here as a reference.
following data, the available amount is limited, partially because the process for creating such data is
time-consuming and less well-defined when human crowd-scouring is considered. Inspired by the
success of recent GPT models in text-annotation tasks [17], we propose to leverage ChatGPT/GPT-4
for multimodal instruction-following data collection, based on the widely existing image-pair data.
For an image Xv and its associated caption Xc, it is natural to create a set of questions Xq with the
intent to instruct the assistant to describe the image content. We prompt GPT-4 to curate such a list
of questions (see details in Appendix). Therefore, a simple way to expand an image-text pair to its
instruction-following version is Human : Xq Xv<STOP> Assistant : Xc<STOP>. Though cheap to
construct, this simple expanded version lacks diversity and in-depth reasoning in both the instructions
and responses.
To mitigate this issue, we leverage language-only GPT-4 or ChatGPT as the strong teacher (both
accept only text as input), to create instruction-following data involving visual content. Specifically,
in order to encode an image into its visual features to prompt a text-only GPT, we use two types of
symbolic representations: (i) Captions typically describe the visual scene from various perspectives;
(ii) Bounding boxes usually localize the objects in the scene, and each box encodes the object concept
and its spatial location. One example is shown in the top block of Table 14.
This symbolic representation allows us to encode the image as an LLM-recognizable sequence. We
use COCO images [31] and generate three types of instruction-following data. One example per type
is shown in the bottom block of Table 14. For each type, we first manually design a few examples.
They are the only human annotations we have during data collection, and are used as seed examples
in in-context-learning to query GPT-4.
• Conversation. We design a conversation between the assistant and a person asking questions
about this photo. The answers are in a tone as if the assistant is seeing the image and answering
the question. A diverse set of questions are asked about the visual content of the image, including
the object types, counting the objects, object actions, object locations, relative positions between
objects. Only questions that have definite answers are considered. Please see Appendix for the
detailed prompt.
• Detailed description. To include a rich and comprehensive description for an image, we create a
list of questions with such an intent. We prompt GPT-4 then curate the list (see detailed prompts
3
and curation process in Appendix). For each image, we randomly sample one question from the
list to ask GPT-4 to generate the detailed description.
• Complex reasoning. The above two types focus on the visual content itself, based on which
we further create in-depth reasoning questions. The answers typically require a step-by-step
reasoning process by following rigorous logic.
We collect 158K unique language-image instruction-following samples in total, including 58K in
conversations, 23K in detailed description, and 77k in complex reasoning, respectively. We ablated
the use of ChatGPT and GPT-4 in our early experiments, and found that GPT-4 consistently provides
higher quality instruction-following data, such as spatial reasoning.
4 Visual Instruction Tuning
4.1 Architecture
The primary goal is to effectively leverage the capabilities of both the pre-trained LLM and visual
model. The network archtecture is illustrated in Figure 1. We choose Vicuna [9] as our LLM fϕ(·)
parameterized by ϕ, as it has the best instruction following capabilities in language tasks among"""

In [35]:
context2 = '''In this short tutorial, we will explore how Hugging Face models can be deployed in a Docker Container and exposed as a web service endpoint.

The service it exposes is a translation service from English to French and French to English.

Why someone would like to do that? Other than to learn about those specific technologies, it is a very convenient way to try and test the thousands of models that exists on Hugging Face, in a clean and isolated environment that can easily be replicated, shared or deployed elsewhere than on your local computer.

In this tutorial, you will learn how to use Docker to create a container with all the necessary code and artifacts to load Hugging Face models and to expose them as web service endpoints using Flask.

All code and configurations used to write this blog post are available in this GitHub Repository. You simply have to clone it and to run the commands listed in this tutorial to replicate the service on your local machine.

Installing Docker
The first step is to install Docker. The easiest way is by simply installing Docker Desktop which is available on MacOS, Windows and Linux.

Creating the Dockerfile
The next step is to create a new Git repository where you will create a Dockerfile. The Dockerfile is where all instructions are written that tells Docker how to create the container.

I would also strongly encourage you to install and use hadolint, which is a really good Docker linter that helps people to follow Docker best practices. There is also a plugin for VS Code if this is what you use as you development IDE.

Base image and key installs
The first thing you define in a Dockerfile is the base image to use to initialize the container. For this tutorial, we will use Ubuntu’s latest LTS:

# Use Ubuntu's current LTS
FROM ubuntu:jammy-20230804

Since we are working to create a Python web service that expose the predictions of a ML model, the next step is to add they key pieces required for the Python service. Let’s make sure that you only include what is necessary to minimize the size, and complexity, of the container as much as possible:

# Make sure to not install recommends and to clean the 
# install to minimize the size of the container as much as possible.
RUN apt-get update && \
    apt-get install --no-install-recommends -y python3=3.10.6-1~22.04 && \
    apt-get install --no-install-recommends -y python3-pip=22.0.2+dfsg-1ubuntu0.3 && \
    apt-get install --no-install-recommends -y python3-venv=3.10.6-1~22.04 && \
    apt-get clean && \
    rm -rf /var/lib/apt/lists/*

This instruct Docker to install Python3, pip and venv. It also ensures that apt get cleaned of cached files, that nothing more is installed and that we define the exact version of the package we want to install. That is to ensure that we minimize the size of the container, while making sure that the container can easily be reproduced, with the exact same codebase, any time in the future.

Another thing to note: we run multiple commands with a single RUN instruction by piping them together with &&. This is to minimize the number of layers created by Docker for the container, and this is a best practice to follow when creating containers. If you don’t do this and run hadolint, then you will get warning suggesting you to refactor your Dockerfile accordingly.

Copy required files
Now that the base operating system is installed, the next step is to install all the requirements of the Python project we want to deploy in the container:

# Set the working directory within the container
WORKDIR /app

# Copy necessary files to the container
COPY requirements.txt .
COPY main.py .
COPY download_models.py .

First we define the working directory with the WORKDIR instruction. From now on, every other instruction will run from that directory in the container. We copy the local files: requirements.txt, main.py and download_models.py to the working directory.

Create virtual environment
Before doing anything with those files, we are better creating a virtual environment where to install all those dependencies. Some people may wonder why we create an environment within an environment? It is further isolation between the container and the Python application to make sure that there is no possibility of dependencies clashes. This is a good best practice to adopt.

# Create a virtual environment in the container
RUN python3 -m venv .venv

# Activate the virtual environment
ENV PATH="/app/.venv/bin:$PATH"

Install application requirements
Once the virtual environment is created and activated in the container, the next step is to install all the required dependencies in that new environment:

    # Install Python dependencies from the requirements file
RUN pip install --no-cache-dir -r requirements.txt && \
    # Get the models from Hugging Face to bake into the container
    python3 download_models.py

It runs pip install to install all the dependencies listed in requirements.txt. The dependencies are:

transformers==4.30.2
flask==2.3.3
torch==2.0.1
sacremoses==0.0.53
sentencepiece==0.1.99

Just like the Ubuntu package version, we should (have to!) pin (specify) the exact version of each dependency. This is the best way to ensure that we can reproduce this environment any time in the future and to prevent unexpected crashes because code changed in some downstream dependencies that causes issues with the code.

Downloading all models in the container
As you can see in the previous RUN command, the next step is to download all models and tokenizers in the working directory such that we bake the model’s artifacts directly in the container. That will ensures that we minimize the time it takes to initialize a container. We spend the time to download all those artifacts at build time instead of run time. The downside is that the containers will be much bigger depending on the models that are required.

The download_models.py file is a utility file used to download the Hugging Face models used by the service directly into the container. The code simply download the models and tokenizer files from Hugging Face and save them locally (in the working directory of the container):

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import os

def download_model(model_path, model_name):
    """Download a Hugging Face model and tokenizer to the specified directory"""
    # Check if the directory already exists
    if not os.path.exists(model_path):
        # Create the directory
        os.makedirs(model_path)

    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

    # Save the model and tokenizer to the specified directory
    model.save_pretrained(model_path)
    tokenizer.save_pretrained(model_path)

# For this demo, download the English-French and French-English models
download_model('models/en_fr/', 'Helsinki-NLP/opus-mt-en-fr')
download_model('models/fr_en/', 'Helsinki-NLP/opus-mt-fr-en')

Creating the Flask translation web service endpoint
The last thing we have to do with the Dockerfile is to expose the port where the web service will be available and to tell the container what to run when it starts:

# Make port 6000 available to the world outside this container
EXPOSE 6000

ENTRYPOINT [ "python3" ]

# Run main.py when the container launches
CMD [ "main.py" ]

We expose the port 6000 to the outside world, and we tell Docker to run the python3 command with main.py. The main.py file is a very simple file that register the web service’s path using Flask, and that makes the predictions (translations in this case):

from flask import Flask, request, jsonify
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

def get_model(model_path):
    """Load a Hugging Face model and tokenizer from the specified directory"""
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
    return model, tokenizer

# Load the models and tokenizers for each supported language
en_fr_model, en_fr_tokenizer = get_model('models/en_fr/')
fr_en_model, fr_en_tokenizer = get_model('models/fr_en/')

app = Flask(__name__)

def is_translation_supported(from_lang, to_lang):
    """Check if the specified translation is supported"""
    supported_translations = ['en_fr', 'fr_en']
    return f'{from_lang}_{to_lang}' in supported_translations

@app.route('/translate/<from_lang>/<to_lang>/', methods=['POST'])
def translate_endpoint(from_lang, to_lang):
    """Translate text from one language to another. This function is 
    called when a POST request is sent to /translate/<from_lang>/<to_lang>/"""
    if not is_translation_supported(from_lang, to_lang):
        return jsonify({'error': 'Translation not supported'}), 400

    data = request.get_json()
    from_text = data.get(f'{from_lang}_text', '')

    if from_text:
        model = None
        tokenizer = None

        match from_lang:
            case 'en':        
                model = en_fr_model
                tokenizer = en_fr_tokenizer
            case 'fr':
                model = fr_en_model
                tokenizer = fr_en_tokenizer

        to_text = tokenizer.decode(model.generate(tokenizer.encode(from_text, return_tensors='pt')).squeeze(), skip_special_tokens=True)

        return jsonify({f'{to_lang}_text': to_text})
    else:
        return jsonify({'error': 'Text to translate not provided'}), 400
    
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=6000, debug=True)

Building the container
Now that the Dockerfile is completed, the next step is to use it to have Docker to build the actual image of the container. This is done using this command in the terminal:

docker build -t localbuild:en_fr_translation_service .

Note that we specified a tag to make it easier to manage it in between all the other images that may exists in the environment. The output of the terminal will show every step defined in the Dockerfile, and the processing for each of those step. The final output looks like:'''

In [7]:
from anthropic import Anthropic
tokenizer = Anthropic().get_tokenizer()
tokenizer.encode(str(context))

Encoding(num_tokens=3122, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])

In [39]:
context_list = context.split('.')
len(context)

13703

In [96]:
import runpod

runpod.api_key = "YOUR_API_KEY"

endpoint = runpod.Endpoint("YOUR_ENDPOINT_ID")

run_request = endpoint.run(
    {"input": {
        "context": context_list,
        "instruction": "You are a question answering bot who uses the provided context to answer a question\n",
        "question": "How many unique language-image instruction-following samples in total?",
        "target_tokens": 500,
    }
})

In [97]:
%%time 
print(run_request.status())
response = run_request.output(timeout=60)
print(response)

IN_PROGRESS
{'compressed_prompt': 'You are a question answering bot who uses the provided context to answer a question\n\n\nruction tuning large models () using machine-generated\ninstruction-ing data has been to improveshot capabilities on\nnew tasks, but the idea is less explored in theal field\n\n We present the\nfirst attempt to use language-onlyPT-4 to generateal language-image\nructioning data\n\n Our experiments show that LLaVA demonstrates impressiveal chat abilities, sometimes exhibiting the behaviors\nofalPT-4 on unseen images/ions, and yields a 85\n\n1% relative score compared with-4 on a synthetical instructioning\net\n\n One of the core aspirations in intelligence\nis to develop a generalpurpose assistant that can effectively follow-al vision-and-language\nions, with human intent to complete variousworld tasks in the wild [4, 27, 26]\n\n Alpaca [48], Vic [9], GPT-4-LLM [38]\n37th Conference on Neural Systems (IPS 2023)\n\nCV] 11 Dec 2023\nutilize various machine-generated-

In [98]:
from pprint import pprint
pprint(response)

{'compressed_prompt': 'You are a question answering bot who uses the provided '
                      'context to answer a question\n'
                      '\n'
                      '\n'
                      'ruction tuning large models () using machine-generated\n'
                      'instruction-ing data has been to improveshot '
                      'capabilities on\n'
                      'new tasks, but the idea is less explored in theal '
                      'field\n'
                      '\n'
                      ' We present the\n'
                      'first attempt to use language-onlyPT-4 to generateal '
                      'language-image\n'
                      'ructioning data\n'
                      '\n'
                      ' Our experiments show that LLaVA demonstrates '
                      'impressiveal chat abilities, sometimes exhibiting the '
                      'behaviors\n'
                      'ofalPT-4 on unseen images/ions, and yields a 

In [1]:
import boto3
bedrock = boto3.client('bedrock-runtime')

In [85]:
from langchain.llms.bedrock import Bedrock

llm = Bedrock(
    model_id="anthropic.claude-v2",
    client=bedrock,
    model_kwargs={
    "temperature":0.0,
    "max_tokens_to_sample":50,
    "top_k":5
    }
)

In [99]:
question = "How many unique language-image instruction-following samples in total?"
prompt = f'''You are a question answering bot who uses ONLY the provided context <context>{context}<context> 
to answer a question in one concise sentence. <question>{question}<question>. If there is no answer, please write "No answer"'''

In [106]:
%%time
without_compression = llm(prompt)

CPU times: total: 0 ns
Wall time: 3.5 s


In [101]:
prompt_compressed = f'''You are a question answering bot who uses ONLY the provided context 
<context>{response["compressed_prompt"].replace("You are a question answering bot who uses the provided context to answer a question", '')}<context> 
to answer a question in one concise sentence. <question>{question}<question>. If there is no answer, please write "No answer"'''

In [102]:
%%time
with_compression = llm(prompt_compressed)

CPU times: total: 0 ns
Wall time: 1.41 s


In [107]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
scores = scorer.score(without_compression.strip(), with_compression.strip())
scores


rouge_scorer.py     :83   2024-04-06 15:50:53,625 Using default tokenizer.


{'rougeL': Score(precision=1.0, recall=0.3, fmeasure=0.4615384615384615)}

In [104]:
from pprint import pprint
pprint(with_compression)

' 158K unique language-image instruction-following samples in total.'


In [108]:
pprint(without_compression)

(' Based on the provided context, there are 158K unique language-image '
 'instruction-following samples collected in total, including 58K in '
 'conversations, 23K in detailed description, and 77K in complex reasoning.')
